<a href="https://colab.research.google.com/github/avdhoot001/resume_parser/blob/main/final_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#you can import data from your local machine through a zip file from here


In [6]:
from google.colab import files
uploading = files.upload()

Saving resume_data.zip to resume_data.zip


#unzip the zip file

In [7]:
import patoolib
patoolib.extract_archive("/content/resume_data.zip")

INFO patool: Extracting /content/resume_data.zip ...
INFO:patool:Extracting /content/resume_data.zip ...
INFO patool: running /usr/bin/7z x -o./Unpack_3_sdos6u -- /content/resume_data.zip
INFO:patool:running /usr/bin/7z x -o./Unpack_3_sdos6u -- /content/resume_data.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... /content/resume_data.zip extracted to `data'.
INFO:patool:... /content/resume_data.zip extracted to `data'.


'data'

#obtain the gemini api key from google ai.google.dev website

In [4]:
genai.configure(api_key="")

#install required libraries, models, dependancies

In [3]:
!pip install -q -U google-generativeai
!pip install patool
!pip install PyMuPDF
!pip install python-docx
!pip install pytesseract
!pip install Pillow

import pytesseract
!apt-get install tesseract-ocr
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

model = genai.GenerativeModel('gemini-1.5-flash')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


#get response

In [5]:
import os
import re
import fitz  # PyMuPDF
import docx
from PIL import Image
import pytesseract
from difflib import SequenceMatcher

def convert_image_to_text(image_path):
    """Convert image to text using Tesseract OCR."""
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def convert_pdf_to_text(pdf_path):
    """Convert PDF to text using PyMuPDF and Tesseract OCR for images."""
    pdf_document = fitz.open(pdf_path)
    text = ""

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        # Try to extract text using PyMuPDF
        page_text = page.get_text()
        if page_text.strip():
            text += page_text
        else:
            # Fall back to OCR if text extraction fails
            pix = page.get_pixmap()
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            page_text = pytesseract.image_to_string(img)
            text += page_text

    return text

def convert_docx_to_text(docx_path):
    """Convert DOCX to text using python-docx."""
    doc = docx.Document(docx_path)
    text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
    return text



def extract_details_from_text(text):
    prompt = (
        f"Please extract the following details from the given resume text in json format:\n\n"
        f"{text}\n\n"
        "Format your response exactly as follows:\n"
        """
        {
          "name": "",
          "mobileNumber": "",
          "emailId": "",
          "dob": "",
          "summary": "",
          "location": "",
          "skills": [],
          "education": [
            {
              "education": "",
              "instituteName": "",
              "courseName": "",
              "startYear": "",
              "endYear": ""
            }
          ],
          "experience": [
            {
              "companyName": "",
              "jobTitle": "",
              "startDate": "",
              "endDate": ""
            }
          ]
        }
        """

    )
    response = model.generate_content(prompt)
    return response
# resume_details = [extract_details_from_text(text) for text in resume_texts]


#submit the path to your resume here

In [8]:
resume_text = (convert_pdf_to_text("/content/data/Resume1.pdf"))
resume_details = extract_details_from_text(resume_text)
to_markdown(resume_details.text)


> ```json
>         {
>           "name": "ABHINESH NARESH JADHAV",
>           "mobileNumber": "9765430426 /7620379472",
>           "emailId": "abhinesh424@gmail.com",
>           "dob": "07/10/1996",
>           "summary": "Energetic culinary professional with a blend of creativity, passion for food and exceptional cooking skills. Detailed oriented with good communication skills. Array of Knowledge when it comes to menu specialties, food presentation and handling, functionality in a variety of cooking and eating venues. With a strong focus on enhancing the customers dining experience, looking to build hospitality talent with fresh opportunities",
>           "location": "Chitra 204, Gagan Vihar Complex, Achole Road, Nallasopara East 401209",
>           "skills": [
>             "Basic Knowledge of Computer",
>             "Management skills",
>             "Leadership Training",
>             "Self-Motivated",
>             "Professionalism & Strong work ethic",
>             "Critical thinking & Problem solving",
>             "Teamwork & Collaboration"
>           ],
>           "education": [
>             {
>               "education": "SSC",
>               "instituteName": "Maharashtra Board",
>               "courseName": "",
>               "startYear": "",
>               "endYear": "2012"
>             },
>             {
>               "education": "HSC",
>               "instituteName": "Maharashtra Board",
>               "courseName": "",
>               "startYear": "",
>               "endYear": "2014"
>             }
>           ],
>           "experience": [
>             {
>               "companyName": "Ambassador Sky Chef",
>               "jobTitle": "Apprentice",
>               "startDate": "1st Sep 2016",
>               "endDate": "31st Aug 2018"
>             },
>             {
>               "companyName": "Ambassador Sky Chef",
>               "jobTitle": "Commi 3",
>               "startDate": "3 Sep 2018",
>               "endDate": "2 Sep 2019"
>             },
>             {
>               "companyName": "Miniya Turk",
>               "jobTitle": "Commi 2",
>               "startDate": "20 Jan 2020",
>               "endDate": "14 Feb 2022"
>             },
>             {
>               "companyName": "Mladiamante Foods & Beverages Pvt. Ltd.",
>               "jobTitle": "Commi 2",
>               "startDate": "15 June 2022",
>               "endDate": "10 Oct 2022"
>             },
>             {
>               "companyName": "JMJ Ganpatiji Maharaja Hospitality Pvt. Ltd.",
>               "jobTitle": "Commi 1",
>               "startDate": "01 March 2023",
>               "endDate": "till date"
>             }
>           ]
>         }
> ```